# Create Variables at Maximum UH location data (``current_conus1_varsatmaxUH.nc``)
## Version: During Current and Future Climate
### Requires: interpolated variable storm patches (e.g., ``current_conus1_tk_20100510.nc``)

First, import relevant packages.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from ncar_jobqueue import NCARCluster
from dask.distributed import Client

In [2]:
#--------------------------------------------------

#if __name__== "__main__":

#start dask workers
cluster = NCARCluster(memory="109GB", cores=36)
cluster.adapt(minimum=1, maximum=10, wait_count=60)
cluster
#print scripts
print(cluster.job_script())
#start client
client = Client(cluster)
client

#--------------------------------------------------

#!/usr/bin/env bash

#PBS -N dask-worker
#PBS -q regular
#PBS -A P54048000
#PBS -l select=1:ncpus=36:mem=109GB
#PBS -l walltime=01:00:00
#PBS -e /glade/scratch/molina/
#PBS -o /glade/scratch/molina/
JOB_ID=${PBS_JOBID%%.*}



/glade/work/molina/miniconda3/envs/python-tutorial/bin/python -m distributed.cli.dask_worker tcp://10.148.10.15:33495 --nthreads 36 --memory-limit 109.00GB --name dask-worker--${JOB_ID}-- --death-timeout 60 --local-directory /glade/scratch/molina --interface ib0



/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://10.148.10.15:33495 Dashboard: https://jupyterhub.ucar.edu/ch/user/molina/proxy/41815/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


First, do current climate.

In [3]:
which_climate = 'current'

date_range = pd.date_range('2000-10-01','2013-09-30',freq='MS')[(pd.date_range('2000-10-01','2013-09-30',freq='MS').month==12)|
                                                                (pd.date_range('2000-10-01','2013-09-30',freq='MS').month==1)|
                                                                (pd.date_range('2000-10-01','2013-09-30',freq='MS').month==2)|
                                                                (pd.date_range('2000-10-01','2013-09-30',freq='MS').month==3)|
                                                                (pd.date_range('2000-10-01','2013-09-30',freq='MS').month==4)|
                                                                (pd.date_range('2000-10-01','2013-09-30',freq='MS').month==5)]

In [5]:
data_prd = []
data_var_tk1 = []; data_var_tk3 = []; data_var_tk5 = []; data_var_tk7 = []
data_var_eu1 = []; data_var_eu3 = []; data_var_eu5 = []; data_var_eu7 = []
data_var_ev1 = []; data_var_ev3 = []; data_var_ev5 = []; data_var_ev7 = []
data_var_pr1 = []; data_var_pr3 = []; data_var_pr5 = []; data_var_pr7 = []
data_var_qv1 = []; data_var_qv3 = []; data_var_qv5 = []; data_var_qv7 = []


for date in date_range:

    data_uh = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/current_conus1_{date.strftime('%Y%m')}*.nc",
                                parallel=True,combine='by_coords')
    data_tk = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/current_conus1_tk_{date.strftime('%Y%m')}*.nc",
                                parallel=True,combine='by_coords')
    data_eu = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/current_conus1_eu_{date.strftime('%Y%m')}*.nc",
                                parallel=True,combine='by_coords')
    data_ev = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/current_conus1_ev_{date.strftime('%Y%m')}*.nc",
                                parallel=True,combine='by_coords')
    data_pr = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/current_conus1_p_{date.strftime('%Y%m')}*.nc",
                                parallel=True,combine='by_coords')
    data_qv = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/current_conus1_qvapor_{date.strftime('%Y%m')}*.nc",
                                parallel=True,combine='by_coords')
    
    data_uh_values = data_uh.uh_grid.values

    data_var_tk1_values = data_tk.sel(levels=0).var_grid.values
    data_var_tk3_values = data_tk.sel(levels=1).var_grid.values
    data_var_tk5_values = data_tk.sel(levels=2).var_grid.values
    data_var_tk7_values = data_tk.sel(levels=3).var_grid.values

    data_var_eu1_values = data_eu.sel(levels=0).var_grid.values
    data_var_eu3_values = data_eu.sel(levels=1).var_grid.values
    data_var_eu5_values = data_eu.sel(levels=2).var_grid.values
    data_var_eu7_values = data_eu.sel(levels=3).var_grid.values

    data_var_ev1_values = data_ev.sel(levels=0).var_grid.values
    data_var_ev3_values = data_ev.sel(levels=1).var_grid.values
    data_var_ev5_values = data_ev.sel(levels=2).var_grid.values
    data_var_ev7_values = data_ev.sel(levels=3).var_grid.values

    data_var_pr1_values = data_pr.sel(levels=0).var_grid.values
    data_var_pr3_values = data_pr.sel(levels=1).var_grid.values
    data_var_pr5_values = data_pr.sel(levels=2).var_grid.values
    data_var_pr7_values = data_pr.sel(levels=3).var_grid.values

    data_var_qv1_values = data_qv.sel(levels=0).var_grid.values
    data_var_qv3_values = data_qv.sel(levels=1).var_grid.values
    data_var_qv5_values = data_qv.sel(levels=2).var_grid.values
    data_var_qv7_values = data_qv.sel(levels=3).var_grid.values
    
    for i in range(0,data_uh.uh_grid.starttime.shape[0]):
        
        d,s = np.where(data_uh_values[i]==data_uh_values[i].max())

        data_prd.append(data_uh_values[i,d,s][0])

        data_var_tk1.append(data_var_tk1_values[i,d,s][0])
        data_var_tk3.append(data_var_tk3_values[i,d,s][0])
        data_var_tk5.append(data_var_tk5_values[i,d,s][0])
        data_var_tk7.append(data_var_tk7_values[i,d,s][0])

        data_var_eu1.append(data_var_eu1_values[i,d,s][0])
        data_var_eu3.append(data_var_eu3_values[i,d,s][0])
        data_var_eu5.append(data_var_eu5_values[i,d,s][0])
        data_var_eu7.append(data_var_eu7_values[i,d,s][0])

        data_var_ev1.append(data_var_ev1_values[i,d,s][0])
        data_var_ev3.append(data_var_ev3_values[i,d,s][0])
        data_var_ev5.append(data_var_ev5_values[i,d,s][0])
        data_var_ev7.append(data_var_ev7_values[i,d,s][0])

        data_var_pr1.append(data_var_pr1_values[i,d,s][0])
        data_var_pr3.append(data_var_pr3_values[i,d,s][0])
        data_var_pr5.append(data_var_pr5_values[i,d,s][0])
        data_var_pr7.append(data_var_pr7_values[i,d,s][0])

        data_var_qv1.append(data_var_qv1_values[i,d,s][0])
        data_var_qv3.append(data_var_qv3_values[i,d,s][0])
        data_var_qv5.append(data_var_qv5_values[i,d,s][0])
        data_var_qv7.append(data_var_qv7_values[i,d,s][0])
        

data_prd=np.array(data_prd)

data_var_tk1=np.array(data_var_tk1) 
data_var_tk3=np.array(data_var_tk3)
data_var_tk5=np.array(data_var_tk5) 
data_var_tk7=np.array(data_var_tk7)

data_var_eu1=np.array(data_var_eu1)
data_var_eu3=np.array(data_var_eu3)
data_var_eu5=np.array(data_var_eu5)
data_var_eu7=np.array(data_var_eu7)

data_var_ev1=np.array(data_var_ev1)
data_var_ev3=np.array(data_var_ev3)
data_var_ev5=np.array(data_var_ev5)
data_var_ev7=np.array(data_var_ev7)

data_var_pr1=np.array(data_var_pr1)
data_var_pr3=np.array(data_var_pr3)
data_var_pr5=np.array(data_var_pr5)
data_var_pr7=np.array(data_var_pr7)

data_var_qv1=np.array(data_var_qv1)
data_var_qv3=np.array(data_var_qv3)
data_var_qv5=np.array(data_var_qv5)
data_var_qv7=np.array(data_var_qv7)


'\ndata_prd=np.array(data_prd)\n\ndata_var_tk1=np.array(data_var_tk1) \ndata_var_tk3=np.array(data_var_tk3)\ndata_var_tk5=np.array(data_var_tk5) \ndata_var_tk7=np.array(data_var_tk7)\n\ndata_var_eu1=np.array(data_var_eu1)\ndata_var_eu3=np.array(data_var_eu3)\ndata_var_eu5=np.array(data_var_eu5)\ndata_var_eu7=np.array(data_var_eu7)\n\ndata_var_ev1=np.array(data_var_ev1)\ndata_var_ev3=np.array(data_var_ev3)\ndata_var_ev5=np.array(data_var_ev5)\ndata_var_ev7=np.array(data_var_ev7)\n\ndata_var_pr1=np.array(data_var_pr1)\ndata_var_pr3=np.array(data_var_pr3)\ndata_var_pr5=np.array(data_var_pr5)\ndata_var_pr7=np.array(data_var_pr7)\n\ndata_var_qv1=np.array(data_var_qv1)\ndata_var_qv3=np.array(data_var_qv3)\ndata_var_qv5=np.array(data_var_qv5)\ndata_var_qv7=np.array(data_var_qv7)\n'

In [24]:
data_assemble = xr.Dataset({
    
     'data_prd':(['x'],data_prd),
    
     'data_var_tk1':(['x'],data_var_tk1),
     'data_var_tk3':(['x'],data_var_tk3),
     'data_var_tk5':(['x'],data_var_tk5),
     'data_var_tk7':(['x'],data_var_tk7),

     'data_var_eu1':(['x'],data_var_eu1),
     'data_var_eu3':(['x'],data_var_eu3),
     'data_var_eu5':(['x'],data_var_eu5),
     'data_var_eu7':(['x'],data_var_eu7),
    
     'data_var_ev1':(['x'],data_var_ev1),
     'data_var_ev3':(['x'],data_var_ev3),
     'data_var_ev5':(['x'],data_var_ev5),
     'data_var_ev7':(['x'],data_var_ev7),
    
     'data_var_pr1':(['x'],data_var_pr1),
     'data_var_pr3':(['x'],data_var_pr3),
     'data_var_pr5':(['x'],data_var_pr5),
     'data_var_pr7':(['x'],data_var_pr7),
    
     'data_var_qv1':(['x'],data_var_qv1),
     'data_var_qv3':(['x'],data_var_qv3),
     'data_var_qv5':(['x'],data_var_qv5),
     'data_var_qv7':(['x'],data_var_qv7),
    })

In [25]:
data_assemble

<xarray.Dataset>
Dimensions:       (x: 1387488)
Dimensions without coordinates: x
Data variables:
    data_prd      (x) float32 0.020655306 0.020655306 ... 2.1581452 0.33788928
    data_var_tk1  (x) float32 266.07062 266.07062 ... 282.2286 293.25687
    data_var_tk3  (x) float32 257.00842 257.00842 ... 271.17053 277.27087
    data_var_tk5  (x) float32 251.62991 251.62991 ... 258.7273 266.1087
    data_var_tk7  (x) float32 236.88306 236.88306 ... 243.89136 252.90517
    data_var_eu1  (x) float32 2.8475482 2.8475482 ... 13.593076 4.968054
    data_var_eu3  (x) float32 11.225577 11.225577 ... 9.214704 11.921749
    data_var_eu5  (x) float32 27.758238 27.758238 ... 6.82252 6.1406217
    data_var_eu7  (x) float32 35.781425 35.781425 ... 8.173242 3.0902755
    data_var_ev1  (x) float32 -8.091959 -8.091959 ... -10.026191 5.2486706
    data_var_ev3  (x) float32 -6.7410245 -6.7410245 ... -11.65021 3.6466005
    data_var_ev5  (x) float32 -16.732063 -16.732063 ... -11.358544 6.2537613
    data_va

In [26]:
data_assemble.to_netcdf(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/current_conus1_varsatmaxUH.nc")

Second, do future climate.

In [27]:
data_prd = []
data_var_tk1 = []; data_var_tk3 = []; data_var_tk5 = []; data_var_tk7 = []
data_var_eu1 = []; data_var_eu3 = []; data_var_eu5 = []; data_var_eu7 = []
data_var_ev1 = []; data_var_ev3 = []; data_var_ev5 = []; data_var_ev7 = []
data_var_pr1 = []; data_var_pr3 = []; data_var_pr5 = []; data_var_pr7 = []
data_var_qv1 = []; data_var_qv3 = []; data_var_qv5 = []; data_var_qv7 = []


for date in date_range:

    data_uh = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/future_conus1_{date.strftime('%Y%m')}*.nc",
                                parallel=True,combine='by_coords')
    data_tk = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/future_conus1_tk_{date.strftime('%Y%m')}*.nc",
                                parallel=True,combine='by_coords')
    data_eu = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/future_conus1_eu_{date.strftime('%Y%m')}*.nc",
                                parallel=True,combine='by_coords')
    data_ev = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/future_conus1_ev_{date.strftime('%Y%m')}*.nc",
                                parallel=True,combine='by_coords')
    data_pr = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/future_conus1_p_{date.strftime('%Y%m')}*.nc",
                                parallel=True,combine='by_coords')
    data_qv = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/future_conus1_qvapor_{date.strftime('%Y%m')}*.nc",
                                parallel=True,combine='by_coords')
    
    data_uh_values = data_uh.uh_grid.values

    data_var_tk1_values = data_tk.sel(levels=0).var_grid.values
    data_var_tk3_values = data_tk.sel(levels=1).var_grid.values
    data_var_tk5_values = data_tk.sel(levels=2).var_grid.values
    data_var_tk7_values = data_tk.sel(levels=3).var_grid.values

    data_var_eu1_values = data_eu.sel(levels=0).var_grid.values
    data_var_eu3_values = data_eu.sel(levels=1).var_grid.values
    data_var_eu5_values = data_eu.sel(levels=2).var_grid.values
    data_var_eu7_values = data_eu.sel(levels=3).var_grid.values

    data_var_ev1_values = data_ev.sel(levels=0).var_grid.values
    data_var_ev3_values = data_ev.sel(levels=1).var_grid.values
    data_var_ev5_values = data_ev.sel(levels=2).var_grid.values
    data_var_ev7_values = data_ev.sel(levels=3).var_grid.values

    data_var_pr1_values = data_pr.sel(levels=0).var_grid.values
    data_var_pr3_values = data_pr.sel(levels=1).var_grid.values
    data_var_pr5_values = data_pr.sel(levels=2).var_grid.values
    data_var_pr7_values = data_pr.sel(levels=3).var_grid.values

    data_var_qv1_values = data_qv.sel(levels=0).var_grid.values
    data_var_qv3_values = data_qv.sel(levels=1).var_grid.values
    data_var_qv5_values = data_qv.sel(levels=2).var_grid.values
    data_var_qv7_values = data_qv.sel(levels=3).var_grid.values
    

    for i in range(0,data_uh.uh_grid.starttime.shape[0]):
        
        d,s = np.where(data_uh_values[i]==data_uh_values[i].max())

        data_prd.append(data_uh_values[i,d,s][0])

        data_var_tk1.append(data_var_tk1_values[i,d,s][0])
        data_var_tk3.append(data_var_tk3_values[i,d,s][0])
        data_var_tk5.append(data_var_tk5_values[i,d,s][0])
        data_var_tk7.append(data_var_tk7_values[i,d,s][0])

        data_var_eu1.append(data_var_eu1_values[i,d,s][0])
        data_var_eu3.append(data_var_eu3_values[i,d,s][0])
        data_var_eu5.append(data_var_eu5_values[i,d,s][0])
        data_var_eu7.append(data_var_eu7_values[i,d,s][0])

        data_var_ev1.append(data_var_ev1_values[i,d,s][0])
        data_var_ev3.append(data_var_ev3_values[i,d,s][0])
        data_var_ev5.append(data_var_ev5_values[i,d,s][0])
        data_var_ev7.append(data_var_ev7_values[i,d,s][0])

        data_var_pr1.append(data_var_pr1_values[i,d,s][0])
        data_var_pr3.append(data_var_pr3_values[i,d,s][0])
        data_var_pr5.append(data_var_pr5_values[i,d,s][0])
        data_var_pr7.append(data_var_pr7_values[i,d,s][0])

        data_var_qv1.append(data_var_qv1_values[i,d,s][0])
        data_var_qv3.append(data_var_qv3_values[i,d,s][0])
        data_var_qv5.append(data_var_qv5_values[i,d,s][0])
        data_var_qv7.append(data_var_qv7_values[i,d,s][0])
        
        
data_prd=np.array(data_prd)

data_var_tk1=np.array(data_var_tk1) 
data_var_tk3=np.array(data_var_tk3)
data_var_tk5=np.array(data_var_tk5) 
data_var_tk7=np.array(data_var_tk7)

data_var_eu1=np.array(data_var_eu1)
data_var_eu3=np.array(data_var_eu3)
data_var_eu5=np.array(data_var_eu5)
data_var_eu7=np.array(data_var_eu7)

data_var_ev1=np.array(data_var_ev1)
data_var_ev3=np.array(data_var_ev3)
data_var_ev5=np.array(data_var_ev5)
data_var_ev7=np.array(data_var_ev7)

data_var_pr1=np.array(data_var_pr1)
data_var_pr3=np.array(data_var_pr3)
data_var_pr5=np.array(data_var_pr5)
data_var_pr7=np.array(data_var_pr7)

data_var_qv1=np.array(data_var_qv1)
data_var_qv3=np.array(data_var_qv3)
data_var_qv5=np.array(data_var_qv5)
data_var_qv7=np.array(data_var_qv7)

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://10.148.9.237:35749'], ('getitem-ece221c01ba68fdc4c1e15e836670e3e', 19, 0, 0)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://10.148.9.237:35749'], ('getitem-ece221c01ba68fdc4c1e15e836670e3e', 29, 0, 0)
NoneType: None
distributed.scheduler - ERROR - Workers don't 

In [28]:
data_assemble = xr.Dataset({
    
     'data_prd':(['x'],data_prd),
    
     'data_var_tk1':(['x'],data_var_tk1),
     'data_var_tk3':(['x'],data_var_tk3),
     'data_var_tk5':(['x'],data_var_tk5),
     'data_var_tk7':(['x'],data_var_tk7),

     'data_var_eu1':(['x'],data_var_eu1),
     'data_var_eu3':(['x'],data_var_eu3),
     'data_var_eu5':(['x'],data_var_eu5),
     'data_var_eu7':(['x'],data_var_eu7),
    
     'data_var_ev1':(['x'],data_var_ev1),
     'data_var_ev3':(['x'],data_var_ev3),
     'data_var_ev5':(['x'],data_var_ev5),
     'data_var_ev7':(['x'],data_var_ev7),
    
     'data_var_pr1':(['x'],data_var_pr1),
     'data_var_pr3':(['x'],data_var_pr3),
     'data_var_pr5':(['x'],data_var_pr5),
     'data_var_pr7':(['x'],data_var_pr7),
    
     'data_var_qv1':(['x'],data_var_qv1),
     'data_var_qv3':(['x'],data_var_qv3),
     'data_var_qv5':(['x'],data_var_qv5),
     'data_var_qv7':(['x'],data_var_qv7),
    })

In [29]:
data_assemble

<xarray.Dataset>
Dimensions:       (x: 1419928)
Dimensions without coordinates: x
Data variables:
    data_prd      (x) float32 0.32868552 0.15313855 ... 0.34631956 0.81121975
    data_var_tk1  (x) float32 271.86005 270.65414 ... 277.71243 277.97382
    data_var_tk3  (x) float32 262.81952 263.9764 ... 269.95248 270.43793
    data_var_tk5  (x) float32 255.26256 256.70453 256.2 ... 259.25604 258.38528
    data_var_tk7  (x) float32 239.60124 241.03265 ... 245.13449 243.93335
    data_var_eu1  (x) float32 -1.0417252 -5.94834 ... 21.604994 26.293783
    data_var_eu3  (x) float32 7.8951883 10.782152 ... 10.454257 2.5942829
    data_var_eu5  (x) float32 11.999904 17.09954 ... 1.1117716 4.206434
    data_var_eu7  (x) float32 11.568342 16.494816 ... 4.7198586 8.084809
    data_var_ev1  (x) float32 0.42749768 2.7395759 ... -11.806699 -14.932037
    data_var_ev3  (x) float32 4.7497225 2.561105 ... -15.87698 -11.862221
    data_var_ev5  (x) float32 2.6357665 2.942088 ... -11.83317 -9.252181
    da

In [30]:
data_assemble.to_netcdf(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/future_conus1_varsatmaxUH.nc")